# 🤖 Telecom X - Parte 2: Prevendo Churn

Este notebook tem como objetivo prever a evasão (churn) de clientes da Telecom X com base nos dados tratados anteriormente.  
Aplicaremos técnicas de machine learning para construir modelos preditivos e interpretar os fatores que mais influenciam a saída de clientes.

---

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

import json
sns.set(style="whitegrid")


In [ ]:
# Carregando dados do JSON
with open('TelecomX_Data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

df = pd.json_normalize(data)
df.head()

In [ ]:
# Remover coluna de ID
df.drop(columns=["customerID"], inplace=True)

# Converter coluna alvo para binária
df = df[df["Churn"].isin(["Yes", "No"])]
df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})

# Garantir tipo numérico
df["account.Charges.Total"] = pd.to_numeric(df["account.Charges.Total"], errors="coerce")

# Definir colunas
target = "Churn"
num_cols = ["customer.tenure", "account.Charges.Monthly", "account.Charges.Total"]
cat_cols = [col for col in df.columns if col not in num_cols + [target]]


In [ ]:
# Proporção da evasão
df[target].value_counts(normalize=True).plot(kind='bar', color=["#69b3a2", "#ff9999"])
plt.title("Distribuição do Churn")
plt.xticks([0, 1], ["Não Evadiu", "Evadiu"], rotation=0)
plt.ylabel("Proporção")
plt.show()

In [ ]:
# Divisão entre treino e teste
X = df.drop(columns=[target])
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42)

print("Treino:", X_train.shape, "Teste:", X_test.shape)


In [ ]:
# Pipeline de transformação
transformer = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_cols)
])

# Pipelines com modelos
pipe_lr = Pipeline(steps=[
    ('preprocess', transformer),
    ('model', LogisticRegression(max_iter=1000))
])

pipe_rf = Pipeline(steps=[
    ('preprocess', transformer),
    ('model', RandomForestClassifier(n_estimators=100, random_state=42))
])


In [ ]:
# Treinar modelos
pipe_lr.fit(X_train, y_train)
pipe_rf.fit(X_train, y_train)

# Previsões
pred_lr = pipe_lr.predict(X_test)
pred_rf = pipe_rf.predict(X_test)

# Avaliação
def avaliar_modelo(y_true, y_pred, nome):
    print(f"\n📊 Avaliação - {nome}")
    print("Acurácia:", accuracy_score(y_true, y_pred).round(2))
    print("Precisão:", precision_score(y_true, y_pred).round(2))
    print("Recall:", recall_score(y_true, y_pred).round(2))
    print("F1-score:", f1_score(y_true, y_pred).round(2))
    print("Matriz de Confusão:")
    print(confusion_matrix(y_true, y_pred))

avaliar_modelo(y_test, pred_lr, "Regressão Logística")
avaliar_modelo(y_test, pred_rf, "Random Forest")


In [ ]:
# Importância das variáveis (Random Forest)
importances = pipe_rf.named_steps['model'].feature_importances_
features = pipe_rf.named_steps['preprocess'].transformers_[1][1].get_feature_names_out(cat_cols).tolist()
features = num_cols + features

imp_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by="Importance", ascending=False)

sns.barplot(data=imp_df.head(10), x="Importance", y="Feature", palette="crest")
plt.title("Top 10 - Importância das Variáveis")
plt.show()

## 🧠 Conclusões

- O modelo de Random Forest apresentou melhor desempenho geral em comparação com a Regressão Logística.
- Variáveis como `tenure`, `Contract`, `InternetService` e `PaymentMethod` foram altamente relevantes na predição de churn.

### ✅ Recomendações Estratégicas

- Clientes com contratos mensais e pouco tempo de casa devem ser foco de campanhas de retenção.
- A forma de pagamento também está relacionada à evasão, especialmente "Electronic check".
- Serviços adicionais como suporte técnico e segurança online tendem a aumentar a permanência do cliente.

---